# Onur Poyraz 2016705069

I hereby declare that I observed the honour code of the university when preparing the homework.

## Pr?gr?mm?ng?H?m?w?rk 3

In this exercise we model a string of text using a Markov(1) model. For simplicity we only consider letters 'a-z'. Capital letters 'A-Z' are mapped to the corresponding ones. All remaining letters, symbols, numbers, including spaces, are denoted by '.'.


We have a probability table $T$ where $T_{i,j} = p(x_t = j | x_{t-1} = i)$  transition model of letters in English text for $t=1,2 \dots N$. Assume that the initial letter in a string is always a space denoted as $x_0 = \text{'.'}$. Such a model where the probability table is always the same is sometimes called a stationary model.

1. For a given $N$, write a program to sample random strings with letters $x_1, x_2, \dots, x_N$ from $p(x_{1:N}|x_0)$
1. Now suppose you are given strings with missing letters, where each missing letter is denoted by a question mark (or underscore, as below). Implement a method, that samples missing letters conditioned on observed ones, i.e., samples from $p(x_{-\alpha}|x_{\alpha})$ where $\alpha$ denotes indices of observed letters. For example, if the input is 't??.', we have $N=4$ and
$x_1 = \text{'t'}$ and $x_4 = \text{'.'}$, $\alpha=\{1,4\}$ and $-\alpha=\{2,3\}$. Your program may possibly generate the strings 'the.', 'twi.', 'tee.', etc. Hint: make sure to make use all data given and sample from the correct distribution. Implement the method and print the results for the test strings below. 
1. Describe a method for filling in the gaps by estimating the most likely letter for each position. Hint: you need to compute
$$
x_{-\alpha}^* = \arg\max_{x_{-\alpha}} p(x_{-\alpha}|x_{\alpha})
$$
Implement the method and print the results for the following test strings along with the log-probability  $\log p(x_{-\alpha}^*,x_{\alpha})$.
1. Discuss how you can improve the model to get better estimations.

In [1]:
test_strings = ['th__br__n.f_x.', '_u_st__n_.to_be._nsw_r__','i__at_._a_h_n_._e_r_i_g','q___t.___z._____t.__.___.__.']

Hint: The code below loads a table of transition probabilities for English text.

In [2]:
import csv
from IPython.display import display, Latex, Math
import numpy as np
import matplotlib.pyplot as plt


alphabet = [chr(i+ord('a')) for i in range(26)]
alphabet.append('.')
letter2idx = {c:i for i,c in enumerate(alphabet)}

T = []
with open('transitions.csv') as csvfile:
    reader = csv.reader(csvfile, delimiter=',')
    for row in reader:
        T.append(row)

print('Example')
## p(x_t = 'u' | x_{t-1} = 'q')
display(Latex(r"$p(x_t = \text{'u'} | x_{t-1} = \text{'q'})$"))
print (T[letter2idx['q']][letter2idx['u']])
display(Latex(r"$p(x_t | x_{t-1} = \text{'a'})$"))
for c,p in zip(alphabet,T[letter2idx['a']]):
    print(c,p)

Example


<IPython.core.display.Latex object>

0.9949749


<IPython.core.display.Latex object>

a 0.0002835
b 0.0228302
c 0.0369041
d 0.0426290
e 0.0012216
f 0.0075739
g 0.0171385
h 0.0014659
i 0.0372661
j 0.0002353
k 0.0110124
l 0.0778259
m 0.0260757
n 0.2145354
o 0.0005459
p 0.0195213
q 0.0001749
r 0.1104770
s 0.0934290
t 0.1317960
u 0.0098029
v 0.0306574
w 0.0088799
x 0.0009562
y 0.0233701
z 0.0018701
. 0.0715219


Firstly I took T lists as a matrix Data and I normalize the data because it is not completely normalised. (I see that some of the rows sum is not equal to 1 exactly. Some of them is equal to 1.01 and some of them are equals to 0.99)

In [3]:
from sklearn.preprocessing import normalize
Data_load = np.loadtxt("transitions.txt",dtype=float)
Data = normalize(Data_load, axis=1, norm='l1')
print (Data[26,:])

[1.062437e-01 4.445020e-02 3.916000e-02 2.829470e-02 2.130840e-02
 4.007930e-02 1.717830e-02 6.060470e-02 6.781650e-02 3.466000e-03
 4.545100e-03 2.430190e-02 4.064290e-02 2.348820e-02 6.499200e-02
 2.734980e-02 2.220800e-03 2.140680e-02 7.046870e-02 1.460781e-01
 9.239900e-03 7.949700e-03 6.063850e-02 1.107000e-04 1.146380e-02
 2.911000e-04 5.621020e-02]


## Part 1

In this part I randomly choose letters over probablity distributions according to previous letter.

In [4]:
N = 100
text='.'

for k in range(N):
    index=alphabet.index(text[k])
    output = np.random.choice(alphabet,1, p=Data[index,:])
    text=text+output[0]
print (text)

.it.in.istanghisthom.s.m.y.ig.tr.het.inc.st.s.hthemesan.thenoun.ds.hen..hesindabllore.emasheny.grount


## Part 2

Here I am try to estimate the missing letters according to first previous and first later non-missing letter. I am randomly choose missing letters over probability distributions.

In [5]:
for i in range(len(test_strings)):
    test = '.' + test_strings[i] + '.'
    text = list(test)
    k = 0
    for j in range(k,len(text)):
        if text[j]=='_':
            first_letter = text[j-1]
            first_index=alphabet.index(first_letter)
            count = 0
            for k in range(j+1,len(text)):
                if text[k] == '_':
                    count = count + 1
                else:
                    last_letter = text[k]
                    last_index = alphabet.index(last_letter)
                    break
            for l in range(j,k):
                first_element = np.dot(Data[first_index,:],np.linalg.matrix_power(Data,l+count+1-k))
                second_element = np.dot(np.linalg.matrix_power(Data,k-l-1),Data[:,last_index])
                probability = np.multiply(first_element,second_element.T)/np.dot(first_element,second_element)
                text[l] = np.random.choice(alphabet,1, p=probability)[0]
    print ("".join(text))

.the.brdon.fex..
.futstheng.toobe.insweror.
.ioeath.pathans.ceerting.
.qusat.yerz..e.int.he.agd.io..


## Part 3

In this part I made almost everyhing same as previous part. I just did not pick missing letters randomly. I pick letters that maximize the probability over probability table.

In [6]:
for i in range(len(test_strings)):
    test = '.' + test_strings[i] + '.'
    text = list(test)
    for j in range(len(text)):
        if text[j]=='_':
            first_letter = text[j-1]
            first_index=alphabet.index(first_letter)
            count = 0
            for k in range(j+1,len(text)):
                if text[k] == '_':
                    count = count + 1
                else:
                    last_letter = text[k]
                    last_index = alphabet.index(last_letter)
                    break
            for l in range(j,k):
                first_element = np.dot(Data[first_index,:],np.linalg.matrix_power(Data,l+count+1-k))
                second_element = np.dot(np.linalg.matrix_power(Data,k-l-1),Data[:,last_index])
                probability = np.multiply(first_element,second_element.T)/np.dot(first_element,second_element)
                choise = np.argmax(probability)
                text[l] = alphabet[choise]
    print ("".join(text))

.the.br.an.fex..
.oursthand.to.be.answeres.
.in.ath.wathend.he.r.ing.
.qur.t.t.rz.t....t.ae.t.e.ae..


## Part 4

We can use such systems that calculate more than one letter before missing letter and also more than one letter after missing letter. Also we have to analyze lower case and upper case letters as well. It is of course hard to programming but will be much more efficient. One another problem with this data is that sometimes data can stuck at some part of it. For example for last test string there is 4 '.' because when first letter equals to '.' and last letter equals to 't' it stucks at that point and generate all letter as '.'